In [19]:
import pandas as pd
import numpy as np
import csv
import math
from bs4 import BeautifulSoup
from gensim.summarization.bm25 import get_bm25_weights
from rank_bm25 import BM25Okapi
import json
from termcolor import colored
import re
import itertools
from functools import reduce

In [55]:
with open('./output_data/tmp/article_pool.json', 'r') as input_file:
    phrase_list = json.load(input_file)

data_df_layer_1 = pd.DataFrame(phrase_list[0])
layer_num = len(phrase_list)
unique_phrase_list = np.unique(data_df_layer_1['phrase'].values)

In [60]:
unique_phrase_list

array(['band easy access', 'band mouth', 'band play', 'band rain',
       'band upper', 'band water', 'easy access mouth',
       'easy access play', 'easy access rain', 'easy access upper',
       'easy access water', 'golf band', 'golf easy access',
       'golf golfer', 'golf lightweight', 'golf mouth', 'golf play',
       'golf rain', 'golf upper', 'golf water', 'golfer band',
       'golfer easy access', 'golfer lightweight', 'golfer mouth',
       'golfer play', 'golfer rain', 'golfer upper', 'golfer water',
       'lightweight band', 'lightweight easy access', 'lightweight mouth',
       'lightweight play', 'lightweight rain', 'lightweight upper',
       'lightweight water', 'mouth play', 'mouth rain', 'mouth upper',
       'mouth water', 'play rain', 'play upper', 'play water',
       'upper rain', 'upper water', 'water rain'], dtype=object)

In [82]:
def p_list_fixed_points(layer_num):
    ans = []
    for i in range(layer_num):
        p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[i])
        for j in range(len(p_list)):
            if p_list[j] not in ans:
                ans.append(p_list[j])
    return ans

In [83]:
p_list_fixed_points(2)

[['golf'],
 ['golfer'],
 ['lightweight'],
 ['band'],
 ['easy access'],
 ['mouth'],
 ['play'],
 ['upper'],
 ['water'],
 ['rain'],
 ['cover', 'covered'],
 ['gas'],
 ['polyethylene'],
 ['central cavity'],
 ['centrally located']]

In [85]:
p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[0])
p_list

[['golf'],
 ['golfer'],
 ['lightweight'],
 ['band'],
 ['easy access'],
 ['mouth'],
 ['play'],
 ['upper'],
 ['water'],
 ['rain']]

In [100]:
p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[1])
p_list

[['golf'],
 ['rain'],
 ['cover', 'covered'],
 ['gas'],
 ['play'],
 ['polyethylene'],
 ['golfer'],
 ['lightweight'],
 ['central cavity'],
 ['centrally located']]

In [131]:
p_list[2]

['cover', 'covered']

In [132]:
b = p_list[2]

In [133]:
a = 'cover and not  ahahhaha'

In [134]:
b[0] in a

True

In [136]:
1 in [c in a for c in b]

[True, False]

In [137]:
np.argmax([c in a for c in b])

0

In [24]:
p_list = p_list_fixed_points(0)
p_list

[]

In [39]:
list_of_phrase_list = []
for i in range(len(unique_phrase_list)):
    tmplist =[]
    for j in range(len(p_list)):
        if p_list[j] in unique_phrase_list[i]:
            tmplist.append(p_list[j])
    list_of_phrase_list.append(tmplist)

TypeError: 'in <string>' requires string as left operand, not list

In [41]:
def create_sentece_list(unique_phrase_list):
    list_of_phrase_list = []
    for i in range(len(unique_phrase_list)):
        tmplist =[]
        for j in range(len(p_list)):
            if p_list[j] in unique_phrase_list[i]:
                tmplist.append(p_list[j])
        list_of_phrase_list.append(tmplist)
    return list_of_phrase_list

In [42]:
def create_sentence_pool(data_df_layer_1):
    article_list = data_df_layer_1['article'].values
    #for each article, find all sentence
    article_list[0]
    sentence_dic = {}
    list_sentence = []
    s_count = 0 #sentence index
    for i in range(len(article_list)):
        #for every sentence, if not in sentence_list, push sentence in list
        tmp_sentence_list = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article_list[i])
        for j in range(len(tmp_sentence_list)):
#          if tmp_sentence_list[j] not in article_list:
            sentence_dic[s_count] = tmp_sentence_list[j]
            list_sentence.append(tmp_sentence_list[j])
            s_count +=1
    return np.unique(list_sentence)

In [43]:
#%%
#create the article pool now
article_list = data_df_layer_1['article'].values
#for each article, find all sentence
sentence_dic = {}
list_sentence = []
s_count = 0 #sentence index
for i in range(len(article_list)):
    #for every sentence, if not in sentence_list, push sentence in list
    tmp_sentence_list = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', article_list[i])
    for j in range(len(tmp_sentence_list)):
#         if tmp_sentence_list[j] not in article_list:
        sentence_dic[s_count] = tmp_sentence_list[j]
        list_sentence.append(tmp_sentence_list[j])        
        s_count +=1
list_sentence = np.unique(list_sentence)


In [44]:
def ven_diagram(list_sentence,unique_phrase_list,p_list):
    #generate the list of list
    list_of_list_com = []
    counter = len(p_list)
    while counter > 0:
        list_of_list_com.append(list(itertools.combinations(p_list,counter)))
        counter-=1
        
    #venn cover
    len_p_list = len(p_list)#len of p_list
    #find the sentence that matches current cover
    ans = []
    for i in range(len(list_of_list_com)):
        for j in range(len(list_of_list_com[i])):
            #find the phrase list that not in com_list
            cur_list = list(list_of_list_com[i][j])
            not_in_list = []
            for pos in range(len(p_list)):
                if p_list[pos] not in cur_list:
                    not_in_list.append(p_list[pos])
            
            #find the sentence that cover all phrase in list_of_list_com[i][j] and not covered in not_in_list
            for p in  range(len(list_sentence)):
                flag_good = True
                flag_good_filter = True
                for q in range(len(cur_list)):
                    if cur_list[q] not in list_sentence[p]:
                        flag_good = False
                        break
                for g in range(len(not_in_list)):
                    if not_in_list[g] in list_sentence[p]:
                        flag_good_filter = False
                        break
                #pass all exam
                #push current setence to result
                if flag_good==True and flag_good_filter==True and list_sentence[p] not in ans:
                    ans.append(list_sentence[p])
                    
        print('combination:',len_p_list,' choose ',len_p_list - i,' current lengh of ans is', len(ans))
                    
    return ans

# a = ven_diagram(list_sentence.copy(),unique_phrase_list,p_list)
# len(a[1])

In [48]:
def main_func():
    with open('./output_data/tmp/article_pool.json', 'r') as input_file:
        phrase_list = json.load(input_file)
    layer_num = len(phrase_list)  #how many layer
    full_output = []
    #<change on 07032019>
    list_phrase = []
    allsentence = []
    #<change on 07032019>
    for i in range(layer_num):
        data_df_layer_1 = pd.DataFrame(phrase_list[i])
        unique_phrase_list = np.unique(data_df_layer_1['phrase'].values)
        #p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[i])
#         list_of_phrase_list =  create_sentece_list(unique_phrase_list)
        p_list = p_list_fixed_points(layer_num)
        #<change on 07032019 >
        list_phrase.append(list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[i]))
        #<change on 07032019 >
        list_sentence = create_sentence_pool(data_df_layer_1)
        #run
        answer = set_cover(list(list_sentence),list(unique_phrase_list),p_list.copy())
        full_output.append(answer)
        #change 07032019
        allsentence += answer
        #change 07032019
        print('current layer is: ',i )
        #annotating_function(answer.copy(),p_list)
        print('length of current p_list ', len(p_list))
        
    #<------change on 0703/2019------------->
    #filter the selected sentence
    
    
    
    #<------change on 0703/2019------------->
    with open('./output_data/summaries.json', 'w') as outfile:
            json.dump(full_output, outfile, indent = 4)

    a =  filter_sentence(allsentence,list_phrase)
    new_annot(a,list_phrase)
#%%
# Runs the entire program



In [117]:
#%%
def set_cover(sentence_list,unique_phrase_list,p_list):
    #at each iteration
        #find the sentence that cover most number of unvisted phrase
        #mark those phrase as visited (pop)
        #mark the sentence as visited (pop)
    answer_sentence_list = []
    touch_count_dic = []
    count = 0
    isempty = False
    while len(p_list) > 0 and isempty==False:
        #create a data structure to save how many unvisited phrase the current sentence touched
        touch_count_dic = {} #key as num of phrase touched, value is a list of index of sentence
        global_max_count = 0
        for i in range(len(sentence_list)):
            #compute num of touched
            tmpcount = 0
            for j in range(len(p_list)):
                if 1 in [c in sentence_list[i] for c in p_list[j]]:
#                 if p_list[j] in sentence_list[i]:
                    tmpcount+=1
    
            if tmpcount > global_max_count:
                global_max_count = tmpcount
            # save current result in dic
            if tmpcount in touch_count_dic:
                #return the list
                curlist = touch_count_dic.get(tmpcount)
                curlist.append(i)
            else:
                tmplist = []
                tmplist.append(i)
                touch_count_dic[tmpcount] = tmplist
        #use the global max count to return lit of index of sentence that lead to the max current count
        
        list_of_max_index_sentence = touch_count_dic[global_max_count]
        #pick the first one
        selected_max_sentence_index = list_of_max_index_sentence[0]
        selected_max_sentence = sentence_list[selected_max_sentence_index]
        #set cover
        visited_list = []
#         print('what is sentence now', selected_max_sentence)
        
        for loc in range(len(p_list)):
            if 1 in [c in selected_max_sentence for c in p_list[loc]] and p_list[loc] not in visited_list:
#             if p_list[loc] in selected_max_sentence and p_list[loc] not in visited_list:
                visited_list.append(p_list[loc])
                
        #delete all visited list
        print('!!!! visted list is', len(visited_list))
        
        
#         print('what is sentence now', selected_max_sentence)
        if len(visited_list) == 0:
            isempty=True
            break
        for pos2 in range(len(visited_list)):
            p_list.remove(visited_list[pos2])
        answer_sentence_list.append(selected_max_sentence)
        #remove the current sentencn
        sentence_list.pop(selected_max_sentence_index)
#         print('length of sentence list is', len(sentence_list))
#         print('len of remainng list', p_list)
    return answer_sentence_list

In [138]:
#%%
def annotating_function(answer,p_list): #only mark the first occurance of a phrase exist in sentence
    #iter through every answer
    for i in range(len(answer)):
        for j in range(len(p_list)):
            if p_list[j] in answer[i]:
                #find the starting index
                start = answer[i].find(p_list[j])
                end = start + len(p_list[j])
                answer[i] = answer[i][0:start] + colored(p_list[j],'red') + answer[i][end:]
    for i in range(len(answer)):
        print('index :', i, '', answer[i] +'\n')


def new_annot(answer,p_list):
    for i in range(len(answer)):
        for j in range(len(p_list[0])):
            #first check if that exist
            if 1 in [c in answer[i] for c in p_list[0][j]]:
                #find the index 
                index = np.argmax([c in answer[i] for c in p_list[0][j]])
#             if p_list[0][j] in answer[i]:
                #find the starting index
                start = answer[i].find(p_list[0][j][index])
                end = start + len(p_list[0][j][index])
                answer[i] = answer[i][0:start] + colored(p_list[0][j][index],'red') + answer[i][end:]
                
        for j in range(len(p_list[1])):
            if 1 in [c in answer[i] for c in p_list[1][j]]:
                #find the index 
                index = np.argmax([c in answer[i] for c in p_list[1][j]])
#             if p_list[0][j] in answer[i]:
                #find the starting index
                start = answer[i].find(p_list[1][j][index])
                end = start + len(p_list[1][j][index])
                answer[i] = answer[i][0:start] + colored(p_list[1][j][index],'green') + answer[i][end:]
                
                
        if len(p_list) >=3:       
            for j in range(len(p_list[2])):
                if 1 in [c in answer[i] for c in p_list[2][j]]:
                    #find the index 
                    index = np.argmax([c in answer[i] for c in p_list[2][j]])
    #             if p_list[0][j] in answer[i]:
                    #find the starting index
                    start = answer[i].find(p_list[2][j][index])
                    end = start + len(p_list[2][j][index])
                    answer[i] = answer[i][0:start] + colored(p_list[2][j][index],'blue') + answer[i][end:]

                
    for i in range(len(answer)):
        print('index :', i, '', answer[i] +'\n')
        

#scoring system
 # of phrases covered
 # of pairs of phrases covered
 # of layers covered

def filter_sentence(full_output,list_phrase):
    final_ans = []
    #save sentence that beencoved from multip layer phrase and sentence that covered by at least 3 phrase in 
    for i in range(len(full_output)):
        #iter through every sentence
        flag_list = [0]*len(list_phrase)
        for j in range(len(list_phrase)):
            #at current 
            for pos in range(len(list_phrase[j])):
                if 1 in [c in full_output[i] for c in list_phrase[j][pos]]:
                #if list_phrase[j][pos] in full_output[i]:
                    flag_list[j] = 1
                    break
        if flag_list.count(1) >=2:
            
            if full_output[i] not in final_ans:
                final_ans.append(full_output[i])
        
        count_list =[0]*len(list_phrase)
        for j2 in range(len(list_phrase)):
            counter = 0
            for pos2 in range(len(list_phrase[j])):
                if 1 in [c in full_output[i] for c in list_phrase[j2][pos2]]:
                #if list_phrase[j2][pos2] in full_output[i]:
                    counter+=1
            count_list[j2] = counter
        
#         print('what is count_list', count_list)
        if np.max(count_list) >=3:
            if full_output[i] not in final_ans:
                final_ans.append(full_output[i])
            
    return final_ans
#%%




In [141]:
def main_func_new():
    with open('./output_data/tmp/article_pool.json', 'r') as input_file:
        phrase_list = json.load(input_file)
    layer_num = len(phrase_list)  #how many layer
    full_output = []
    #<change on 07032019>
    list_phrase = []
    allsentence = []
    #<change on 07032019>
    for i in range(layer_num):
        data_df_layer_1 = pd.DataFrame(phrase_list[i])
        unique_phrase_list = np.unique(data_df_layer_1['phrase'].values)
        #p_list = list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[i])
#         list_of_phrase_list =  create_sentece_list(unique_phrase_list)
        p_list = p_list_fixed_points(layer_num)
        #<change on 07032019 >
        list_phrase.append(list(pd.read_json('./output_data/tmp/selected_phrases.json',typ='series')[i]))
        #<change on 07032019 >
        list_sentence = create_sentence_pool(data_df_layer_1)
        #run
        
    
        answer = set_cover(list(list_sentence),list(unique_phrase_list),p_list.copy())
        full_output.append(answer)
        #change 07032019
        allsentence += answer
        #change 07032019
        print('current layer is: ',i )
        #annotating_function(answer.copy(),p_list)
        print('length of current p_list ', len(p_list))
        
    #<------change on 0703/2019------------->
    #filter the selected sentence
    
    
   
    #<------change on 0703/2019------------->
#     with open('./output_data/summaries.json', 'w') as outfile:
#             json.dump(full_output, outfile, indent = 4)
    
    a =  filter_sentence(allsentence,list_phrase)

    new_annot(a,list_phrase)
    with open('./output_data/summaries.json', 'w') as outfile:
        json.dump(a, outfile, indent = 4)

#%%
# Runs the entire program
a = main_func_new()

!!!! visted list is 4
!!!! visted list is 3
!!!! visted list is 2
!!!! visted list is 1
!!!! visted list is 1
!!!! visted list is 1
!!!! visted list is 1
!!!! visted list is 0
current layer is:  0
length of current p_list  15
!!!! visted list is 4
!!!! visted list is 3
!!!! visted list is 2
!!!! visted list is 2
!!!! visted list is 1
!!!! visted list is 1
!!!! visted list is 1
!!!! visted list is 1
current layer is:  1
length of current p_list  15
index : 0  A band of elastic surrounds and gathers the base and the top portions thereof in pleats, and the sleeve is sufficiently elongated so that when the base is fitted over the mouth of the bag, the upper portion of the cover will fold over the club heads.

index : 1  A flexible, waterproof, lightweight compact cover for the top of a golf bag and for golf clubs having a first end and a second end.

index : 2  A 7-week, 10-session individual training program was implemented with a youth elite football (soccer) player who had been underper